In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import expit

import statsmodels.formula.api as smf
import statsmodels.api as sm
%matplotlib inline


In [39]:
# define confounders
c = np.random.binomial(1, .5, 2000)
c2 = np.random.normal(size=2000)

# define treatment
p = np.exp(.7*c - .1*c2)/(1 + np.exp(.7*c - .1*c2))
trt = np.random.binomial(1, p, 2000)

# outcome
y = .1*trt - .5*c - .2*c2 + np.random.normal(0, .5, 2000)

data = pd.DataFrame({'y': y, 'trt':trt, 'c2': c2, 'c':c})
data.head()

,y,trt,c2,c
0,-0.745213,0,1.009673,0
1,-0.770460,1,0.568863,1
2,-0.127567,1,-0.717571,1
3,0.201624,0,-1.154207,1
4,0.136969,0,-0.945644,0


## Unadjusted Effects

In [40]:
res = smf.glm('y ~ trt', data).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                 2000
Model:                            GLM   Df Residuals:                     1998
Model Family:                Gaussian   Df Model:                            1
Link Function:               identity   Scale:                         0.33320
Method:                          IRLS   Log-Likelihood:                -1737.9
Date:                Tue, 26 Jan 2021   Deviance:                       665.73
Time:                        09:11:02   Pearson chi2:                     666.
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.2122      0.020    -10.551      0.000      -0.252      -0.173
trt            0.0555      0.026      2.117      0.034       0.004       0.107
==============================================================================
"""

## Regression Adjustment

In [41]:
res = smf.glm('y ~ trt + c + c2', data).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                 2000
Model:                            GLM   Df Residuals:                     1996
Model Family:                Gaussian   Df Model:                            3
Link Function:               identity   Scale:                         0.24343
Method:                          IRLS   Log-Likelihood:                -1423.0
Date:                Tue, 26 Jan 2021   Deviance:                       485.89
Time:                        09:11:14   Pearson chi2:                     486.
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0302      0.019     -1.562      0.118      -0.068       0.008
trt            0.1155      0.023      5.067      0.000       0.071       0.160
c             -0.4490      0.022    -20.026      0.000      -0.493      -0.405
c2            -0.2003      0.011    -18.011      0.000      -0.222      -0.179
==============================================================================
"""

## propensity scores

### Stabalization